In [1]:
using CSV
using DataFrames
using ProgressBars

In [2]:
deposit_txs = CSV.read("../data/lighter_complete_deposit_txs.csv", DataFrame)
withdraw_txs = CSV.read("../data/lighter_complete_withdraw_txs.csv", DataFrame)

unique_deposit_addresses = Set(deposit_txs[!, :from_address])
unique_withdraw_addresses = Set(withdraw_txs[!, :recipient_address])

outside_tcash_txs = CSV.read("../data/transactions_between_deposit_and_withdraw_addresses.csv", DataFrame)
address_and_withdraw_df = outside_tcash_txs[!, [:from_address, :to_address]];

In [3]:
function unique_and_permuted(address_and_withdraw_df)
    unique_and_permuted_set = Set(Set([]))
    
    for row in eachrow(address_and_withdraw_df)
        push!(unique_and_permuted_set, Set([row.from_address, row.to_address]))
    end
  
    unique_and_permuted_set
end

function dataframe_from_set_of_sets(set_of_sets)
    df = DataFrame(from_address=[], to_address=[])
    for set in set_of_sets
        push!(df, collect(set))
    end
    df
end

function preprocess_data(address_and_withdraw_df)
    set = filter(x -> length(x) == 2, unique_and_permuted(address_and_withdraw_df))
    dataframe_from_set_of_sets(set)
end

preprocess_data (generic function with 1 method)

In [4]:
clean_addresses_df = preprocess_data(address_and_withdraw_df);

In [5]:
# function is_D_type(address, unique_deposit_addresses, unique_withdraw_addresses)
function is_D_type(address)
    address ∈ unique_deposit_addresses && address ∉ unique_withdraw_addresses
end

# function is_W_type(address, unique_deposit_addresses, unique_withdraw_addresses)
function is_W_type(address)
    address ∉ unique_deposit_addresses && address ∈ unique_withdraw_addresses
end

# function is_DW_type(address, unique_deposit_addresses, unique_withdraw_addresses)
function is_DW_type(address)
    address ∈ unique_deposit_addresses && address ∈ unique_withdraw_addresses
end

is_DW_type (generic function with 1 method)

In [6]:
function is_D_W_tx(from_address, to_address)
    is_D_type(from_address) && is_W_type(to_address)
end

function is_W_D_tx(from_address, to_address)
    is_W_type(from_address) && is_D_type(to_address)
end

function is_D_DW_tx(from_address, to_address)
    is_D_type(from_address) && is_DW_type(to_address)
end

function is_DW_D_tx(from_address, to_address)
    is_DW_type(from_address) && is_D_type(to_address)
end

function is_W_DW_tx(from_address, to_address)
    is_W_type(from_address) && is_DW_type(to_address)
end

function is_DW_W_tx(from_address, to_address)
    is_DW_type(from_address) && is_W_type(to_address)
end

function is_DW_DW_tx(from_address, to_address)
    is_DW_type(from_address) && is_DW_type(to_address)
end 

is_DW_DW_tx (generic function with 1 method)

In [7]:
function create_deposit_and_withdraw_df(address_and_withdraw_df, unique_deposit_addresses, unique_withdraw_addresses)
    
    # D | W
    deposit_and_withdraw_matrix = Matrix{String}
    
    for row in ProgressBar(eachrow(address_and_withdraw_df), printing_delay=0.1)
        if is_D_W_tx(row.from_address, row.to_address) || is_D_DW_tx(row.from_address, row.to_address) || is_DW_W_tx(row.from_address, row.to_address)
            deposit_and_withdraw_matrix = vcat(deposit_and_withdraw_matrix, [row.from_address row.to_address])
            
        elseif is_W_D_tx(row.from_address, row.to_address) || is_W_DW_tx(row.from_address, row.to_address) || is_DW_D_tx(row.from_address, row.to_address)
            deposit_and_withdraw_matrix = vcat(deposit_and_withdraw_matrix, [row.to_address row.from_address])
            
        elseif is_DW_DW_tx(row.from_address, row.to_address)
            deposit_and_withdraw_matrix = vcat(deposit_and_withdraw_matrix, [row.from_address row.to_address; row.to_address row.from_address])
            
        else
            throw("The transaction is not from any of the types: D_W, W_D, D_DW, DW_D, W_DW, DW_W, DW_DW")
            
        end
    end
    
    DataFrame(deposit_and_withdraw_matrix, [:deposit_address, :withdraw_address])
end

create_deposit_and_withdraw_df (generic function with 1 method)

In [8]:
D_W_df = create_deposit_and_withdraw_df(clean_addresses_df, unique_deposit_addresses, unique_withdraw_addresses)[2:end,:];

0.0%┣                                          ┫ 0/11.2k [00:00<-18:-43, -0s/it]
0.0%┣                                        ┫ 1/11.2k [00:01<Inf:Inf, InfGs/it]
0.0%┣                                          ┫ 2/11.2k [00:01<02:37:13, 1it/s]
0.0%┣                                          ┫ 3/11.2k [00:01<01:43:33, 2it/s]
28.5%┣██████████▉                           ┫ 3.2k/11.2k [00:01<00:03, 2.7kit/s]
43.8%┣████████████████▋                     ┫ 4.9k/11.2k [00:01<00:02, 3.8kit/s]
52.6%┣████████████████████                  ┫ 5.9k/11.2k [00:01<00:01, 4.2kit/s]
61.6%┣███████████████████████▍              ┫ 6.9k/11.2k [00:02<00:01, 4.6kit/s]
71.0%┣███████████████████████████           ┫ 8.0k/11.2k [00:02<00:01, 4.9kit/s]
80.1%┣██████████████████████████████▌       ┫ 9.0k/11.2k [00:02<00:00, 5.2kit/s]
87.1%┣█████████████████████████████████     ┫ 9.8k/11.2k [00:02<00:00, 5.4kit/s]
93.6%┣██████████████████████████████████▋  ┫ 10.5k/11.2k [00:02<00:00, 5.5kit/s]
99.7%┣██████████████████████

In [41]:
D_W_df

,deposit_address,withdraw_address
,Any,Any
1,0x0c63d55a244657f5606d62856bd9f1ff227c05f2,0x0e54db73f82bd9fde34ebce53ea83bd197e9044c
2,0xd9ee088c6ca2a90d6f0d059af17c2ec2c908bb0f,0xc73ef94bc339a2cb9a1b67820af46bf47484a1ed
3,0xbf7c205febae32f7874b28b9f371fe522e1fd97a,0xe5b5df72187f7d867973615f5e1144b7a95b495f
4,0xe5b5df72187f7d867973615f5e1144b7a95b495f,0xbf7c205febae32f7874b28b9f371fe522e1fd97a
5,0x09fe8f71f8e14b3d6b6456fbafaaef4a27f042cd,0x35f081bdf4740ffa8a56ff98e4b971fbcb7d82a7
6,0xa8308e994d180ca87c6a784fcb8612dec9ede03d,0x46ba0af6bc60e6fabd9957744c057d031c720ace
7,0xf62e92b2452d8a0fbb2c4b03424d679c86660001,0xf94571dbdff33446dabd17040cd6236b0d2c2545
8,0x865ec62a7f46aab0976ad22573fcf319c3f939ce,0xce91fddab3c544b59ebac665a7635561043a7def
9,0x134b9eab4aa4c1489687c18c10d7338656fde32d,0x68a99f89e475a078645f4bac491360afe255dff1


In [10]:
tornado_addresses = Dict(
    "0xd4b88df4d29f5cedd6857912842cff3b20c8cfa3" => "100 DAI",
    "0xfd8610d20aa15b7b2e3be39b396a1bc3516c7144" => "1000 DAI",
    "0x07687e702b410fa43f4cb4af7fa097918ffd2730" => "10000 DAI",
    "0x23773e65ed146a459791799d01336db287f25334" => "100000 DAI",
    "0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc" => "0.1 ETH",
    "0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936" => "1 ETH",
    "0x910cbd523d972eb0a6f4cae4618ad62622b39dbf" => "10 ETH",
    "0xa160cdab225685da1d56aa342ad8841c3b53f291" => "100 ETH",
    "0xd96f2b1c14db8458374d9aca76e26c3d18364307" => "100 USDC",
    "0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d" => "1000 USDC",
    "0x169ad27a470d064dede56a2d3ff727986b15d52b" => "100 USDT",
    "0x0836222f2b2b24a3f36f98668ed8f0b38d1a872f" => "1000 USDT",
    "0x178169b423a011fff22b9e3f3abea13414ddd0f1" => "0.1 WBTC",
    "0x610b717796ad172b316836ac95a2ffad065ceab4" => "1 WBTC",
    "0xbb93e510bbcd0b7beb5a853875f9ec60275cf498" => "10 WBTC",
    "0x22aaa7720ddd5388a3c0a3333430953c68f1849b" => "5000 cDAI",
    "0x03893a7c7463ae47d46bc7f091665f1893656003" => "50000 cDAI",
    "0x2717c5e28cf931547b621a5dddb772ab6a35b701" => "500000 cDAI",
    "0xd21be7248e0197ee08e0c20d4a96debdac3d20af" => "5000000 cDAI"
    );

In [11]:
function get_tcash_transactions_done(address, tornado_transactions_df, tornado_addresses; transaction_type)
    
    address_field =
        if transaction_type == :deposit "from_address" 
            elseif transaction_type == :withdraw "recipient_address"
            else throw("Transaction type parameter error")
        end
    
    # The number of withdraws is initialized at 1 since the withdraw_transaction of the first argument is always present
    # in the withdrawal data. Also, the count should be 1 if there is no other transaction with the same address.
    
    transactions_dict = Dict()
    
    address_transactions = filter(row -> row[address_field] == address, tornado_transactions_df)
    
    # This for loop counts the number of transactions with the same address. At the end, the total number is returned.
    # The count is done considering that the recipient_address of each of the transactions in the withdraw_transactions_df
    # is the same as the recipient_address of the withdraw_transaction input, and that the timestamp of the rows is earlier
    # than the withdraw_transaction input. 
    # The if clause also filters by the transaction hash, since we don't want to count the same transaction two times.
    
    for row ∈ eachrow(address_transactions)
        if haskey(transactions_dict, tornado_addresses[row.tornado_cash_address])
            push!(transactions_dict[tornado_addresses[row.tornado_cash_address]], (hash=row.hash, timestamp=row.block_timestamp))
        else
            transactions_dict[tornado_addresses[row.tornado_cash_address]] = [(hash=row.hash, timestamp=row.block_timestamp)]
        end
    end

    Dict(address => transactions_dict)
end

get_tcash_transactions_done (generic function with 1 method)

In [12]:
get_tcash_transactions_done("0x0c63d55a244657f5606d62856bd9f1ff227c05f2", deposit_txs, tornado_addresses; transaction_type=:deposit)

Dict{String, Dict{Any, Any}} with 1 entry:
  "0x0c63d55a244657f5606d6… => Dict("100 ETH"=>NamedTuple{(:hash, :timestamp), …

In [13]:
first(D_W_df, 5)

,deposit_address,withdraw_address
,Any,Any
1,0x0c63d55a244657f5606d62856bd9f1ff227c05f2,0x0e54db73f82bd9fde34ebce53ea83bd197e9044c
2,0xd9ee088c6ca2a90d6f0d059af17c2ec2c908bb0f,0xc73ef94bc339a2cb9a1b67820af46bf47484a1ed
3,0xbf7c205febae32f7874b28b9f371fe522e1fd97a,0xe5b5df72187f7d867973615f5e1144b7a95b495f
4,0xe5b5df72187f7d867973615f5e1144b7a95b495f,0xbf7c205febae32f7874b28b9f371fe522e1fd97a
5,0x09fe8f71f8e14b3d6b6456fbafaaef4a27f042cd,0x35f081bdf4740ffa8a56ff98e4b971fbcb7d82a7


In [14]:
unique_deposit_addresses = unique(D_W_df[!, :deposit_address]);

In [15]:
unique_withdraw_addresses = unique(D_W_df[!, :withdraw_address]);

In [16]:
function get_addresses_transactions(addresses, tornado_transactions_df, tornado_addresses; transaction_type)
    addresses_transactions = Dict()
    for address in ProgressBar(addresses, printing_delay=2)
        merge!(addresses_transactions, get_tcash_transactions_done(address, tornado_transactions_df, tornado_addresses; transaction_type=transaction_type))
    end
    addresses_transactions
end

get_addresses_transactions (generic function with 1 method)

In [17]:
const deposit_addresses_tcash_transactions = get_addresses_transactions(unique_deposit_addresses, deposit_txs, tornado_addresses; transaction_type=:deposit)

0.0%┣                                        ┫ 0/7.8k [00:02<-4:-19:-24, -2s/it]
0.0%┣                                         ┫ 1/7.8k [00:02<Inf:Inf, InfGs/it]
1.2%┣▌                                           ┫ 90/7.8k [00:04<05:51, 22it/s]
2.4%┣█                                          ┫ 185/7.8k [00:06<04:11, 30it/s]
3.6%┣█▌                                         ┫ 281/7.8k [00:08<03:36, 35it/s]
4.8%┣██                                         ┫ 377/7.8k [00:10<03:19, 37it/s]
6.1%┣██▋                                        ┫ 473/7.8k [00:12<03:07, 39it/s]
7.3%┣███▏                                       ┫ 569/7.8k [00:14<02:59, 40it/s]
8.5%┣███▊                                       ┫ 664/7.8k [00:16<02:53, 41it/s]
9.8%┣████▏                                      ┫ 762/7.8k [00:18<02:47, 42it/s]
11.0%┣████▋                                     ┫ 858/7.8k [00:20<02:43, 43it/s]
12.3%┣█████▏                                    ┫ 955/7.8k [00:22<02:38, 43it/s]
13.4%┣█████▌                

Dict{Any, Any} with 7782 entries:
  "0x50892e106095f415b4b12… => Dict{Any, Any}("1 ETH"=>NamedTuple{(:hash, :time…
  "0x04d473d70c0de9174d001… => Dict{Any, Any}("10 ETH"=>NamedTuple{(:hash, :tim…
  "0x96de6ffa0894aad16cc4b… => Dict{Any, Any}("0.1 ETH"=>NamedTuple{(:hash, :ti…
  "0xa07f530663171df7a1d63… => Dict{Any, Any}("1 ETH"=>NamedTuple{(:hash, :time…
  "0xea5821a1e8fb2810dac7a… => Dict{Any, Any}("1 ETH"=>NamedTuple{(:hash, :time…
  "0xa3a0a77d44ab389b1bbfd… => Dict{Any, Any}("10 ETH"=>NamedTuple{(:hash, :tim…
  "0xa5616c07451d384030f2b… => Dict{Any, Any}("10 ETH"=>NamedTuple{(:hash, :tim…
  "0xd92029bc5c900d3cfc7e0… => Dict{Any, Any}("1 ETH"=>NamedTuple{(:hash, :time…
  "0x580ad8ff46a96e2657f4b… => Dict{Any, Any}("1 ETH"=>NamedTuple{(:hash, :time…
  "0x476b3bee1dd67939d0f47… => Dict{Any, Any}("0.1 ETH"=>NamedTuple{(:hash, :ti…
  "0xd58d95455ba331a1f43b1… => Dict{Any, Any}("10 ETH"=>NamedTuple{(:hash, :tim…
  "0x0c583364c44bc2f0413e7… => Dict{Any, Any}("1 ETH"=>NamedTuple{(:hash, :

In [18]:
const withdraw_addresses_tcash_transactions = get_addresses_transactions(unique_withdraw_addresses, withdraw_txs, tornado_addresses; transaction_type=:withdraw)

0.0%┣                                        ┫ 0/8.1k [00:02<-4:-28:-30, -2s/it]
0.0%┣                                         ┫ 1/8.1k [00:02<Inf:Inf, InfGs/it]
1.4%┣▋                                          ┫ 111/8.1k [00:04<04:51, 27it/s]
2.7%┣█▏                                         ┫ 217/8.1k [00:06<03:39, 36it/s]
4.1%┣█▊                                         ┫ 332/8.1k [00:08<03:08, 41it/s]
5.5%┣██▍                                        ┫ 445/8.1k [00:10<02:52, 44it/s]
6.9%┣███                                        ┫ 553/8.1k [00:12<02:44, 46it/s]
8.3%┣███▌                                       ┫ 666/8.1k [00:14<02:36, 47it/s]
9.7%┣████▏                                      ┫ 778/8.1k [00:16<02:30, 48it/s]
11.1%┣████▋                                     ┫ 894/8.1k [00:18<02:25, 49it/s]
12.5%┣█████▏                                   ┫ 1.0k/8.1k [00:20<02:20, 50it/s]
13.9%┣█████▊                                   ┫ 1.1k/8.1k [00:22<02:17, 51it/s]
15.4%┣██████▎               

Dict{Any, Any} with 8055 entries:
  "0x50892e106095f415b4b12… => Dict{Any, Any}("10 ETH"=>NamedTuple{(:hash, :tim…
  "0xdf93a32c083207cd1ea00… => Dict{Any, Any}("1 ETH"=>NamedTuple{(:hash, :time…
  "0xd1ccc07177c0c27ab78cf… => Dict{Any, Any}("10 ETH"=>NamedTuple{(:hash, :tim…
  "0x6996c90cedd6b7ef51971… => Dict{Any, Any}("10 ETH"=>NamedTuple{(:hash, :tim…
  "0x1d62ca769fcf94d24484b… => Dict{Any, Any}("0.1 ETH"=>NamedTuple{(:hash, :ti…
  "0xab17da946b4ee971e6cd9… => Dict{Any, Any}("0.1 ETH"=>NamedTuple{(:hash, :ti…
  "0x5d57f2e5f61b484eadc14… => Dict{Any, Any}("1 ETH"=>NamedTuple{(:hash, :time…
  "0xb232e7c376462dcc96004… => Dict{Any, Any}("100 ETH"=>NamedTuple{(:hash, :ti…
  "0x1cbfd11c477bb948742ef… => Dict{Any, Any}("1 ETH"=>NamedTuple{(:hash, :time…
  "0xfed2cb2342f345bab5e55… => Dict{Any, Any}("1 ETH"=>NamedTuple{(:hash, :time…
  "0xd099ff6d9f00ce2d35bd6… => Dict{Any, Any}("10 ETH"=>NamedTuple{(:hash, :tim…
  "0x06b3a0712f26c7e72dc37… => Dict{Any, Any}("0.1 ETH"=>NamedTuple{(:hash,

In [39]:
function filter_deposits(address1, address2)
    
    deposit_transactions = deposit_addresses_tcash_transactions[address1]
    withdraw_transactions = withdraw_addresses_tcash_transactions[address2]
    
    pools = filter(pool -> pool ∈ keys(withdraw_transactions), keys(deposit_transactions))
    
    linked_transactions_dict = Dict()
    
    for pool in pools
        latest_withdraw = maximum(map(w_t -> w_t.timestamp, withdraw_transactions[pool]))
        
        filtered_deposits = filter(d_t -> d_t.timestamp < latest_withdraw, deposit_transactions[pool])
        
        if !isempty(filtered_deposits)
            linked_transactions_dict[pool] = (deposit_hashes = map(d_t -> d_t.hash, filtered_deposits),
                    withdraw_hashes = map(w_t -> w_t.hash, withdraw_transactions[pool]))
        end
    end
    
    Dict((deposit_address=address1, withdraw_address=address2) => linked_transactions_dict)
end

filter_deposits (generic function with 1 method)

In [34]:
d = deposit_addresses_tcash_transactions["0x000000001d94b2612380854e74c32548d3ce4720"]
[t.timestamp for t in d["1 ETH"]]

2-element Vector{String31}:
 "2021-10-22 22:21:14 UTC"
 "2021-10-22 22:56:07 UTC"

In [35]:
w = withdraw_addresses_tcash_transactions["0x000000007cbf74626927365e961cc697ef8fed32"]
[t.timestamp for t in w["1 ETH"]]

1-element Vector{String31}:
 "2020-12-26 10:47:20 UTC"

In [40]:
filter_deposits("0x000000001d94b2612380854e74c32548d3ce4720", "0x000000007cbf74626927365e961cc697ef8fed32")

Dict{NamedTuple{(:deposit_address, :withdraw_address), Tuple{String, String}}, Dict{Any, Any}} with 1 entry:
  (deposit_address = "0x000000001d94b2612380854e74c32548d3ce4720", wi… => Dict()

In [48]:
function first_neightbors_heuristic(deposit_and_withdraw_linked_addresses)
    
    addresses_linked_transactions = Dict()
    
    for row in ProgressBar(eachrow(deposit_and_withdraw_linked_addresses), printing_delay=2)
        merge!(addresses_linked_transactions, filter_deposits(row.deposit_address, row.withdraw_address))
    end
    
    filter(element -> !isempty(last(element)), addresses_linked_transactions)
end     

first_neightbors_heuristic (generic function with 1 method)

In [51]:
d = first_neightbors_heuristic(D_W_df)

0.0%┣                                       ┫ 0/12.4k [00:02<-6:-52:-50, -2s/it]
0.0%┣                                        ┫ 1/12.4k [00:02<Inf:Inf, InfGs/it]
100.0%┣████████████████████████████████████┫ 12.4k/12.4k [00:02<00:00, 5.9kit/s]


Dict{Any, Any} with 5861 entries:
  (deposit_address = "0x32… => Dict{Any, Any}("10 ETH"=>(deposit_hashes = ["0x7…
  (deposit_address = "0xeb… => Dict{Any, Any}("10 ETH"=>(deposit_hashes = ["0xe…
  (deposit_address = "0x9c… => Dict{Any, Any}("10 ETH"=>(deposit_hashes = ["0xe…
  (deposit_address = "0xe2… => Dict{Any, Any}("100 ETH"=>(deposit_hashes = ["0x…
  (deposit_address = "0x92… => Dict{Any, Any}("1 ETH"=>(deposit_hashes = ["0xa6…
  (deposit_address = "0xf3… => Dict{Any, Any}("100 DAI"=>(deposit_hashes = ["0x…
  (deposit_address = "0x06… => Dict{Any, Any}("1 ETH"=>(deposit_hashes = ["0x9b…
  (deposit_address = "0xc4… => Dict{Any, Any}("10 ETH"=>(deposit_hashes = ["0x8…
  (deposit_address = "0x3a… => Dict{Any, Any}("10 ETH"=>(deposit_hashes = ["0x4…
  (deposit_address = "0xb9… => Dict{Any, Any}("1 ETH"=>(deposit_hashes = ["0xb5…
  (deposit_address = "0x22… => Dict{Any, Any}("10 ETH"=>(deposit_hashes = ["0x8…
  (deposit_address = "0x50… => Dict{Any, Any}("0.1 ETH"=>(deposit_hashes = 

In [58]:
t = (deposit_address = "0x32192ec423152d5f2428adcecb1525e4eb366e90", withdraw_address = "0xce7cf36d9b8bcacd4c80024a10aadd8f7d1173a2")

(deposit_address = "0x32192ec423152d5f2428adcecb1525e4eb366e90", withdraw_address = "0xce7cf36d9b8bcacd4c80024a10aadd8f7d1173a2")

In [61]:
d[t]["10 ETH"]

(deposit_hashes = ["0x79524bf15538e277ffe74de2cc7980eb247a65747ef5cc3f0053281d550f6716"], withdraw_hashes = ["0xb528dbdf806625de1819dcbd954d01e844b6de0e989c0ebfa1069289364cc6c2"])

In [62]:
keys(d) |> collect

5861-element Vector{Any}:
 (deposit_address = "0x32192ec423152d5f2428adcecb1525e4eb366e90", withdraw_address = "0xce7cf36d9b8bcacd4c80024a10aadd8f7d1173a2")
 (deposit_address = "0xebfcef1eda60358d7e0e81db5bef89dfaaa5f3f5", withdraw_address = "0xf19958f76f0f40f0a14006b7e22be03ad5eae104")
 (deposit_address = "0x9c1c21ec0e88b8334ebc5d388a310f59eca0e381", withdraw_address = "0x25f2af3b84d6a36d38dda369c8f7e7a7b0258941")
 (deposit_address = "0xe2ca7390e76c5a992749bb622087310d2e63ca29", withdraw_address = "0x000000000cc7e508b4b115e64d71ef374cfb7703")
 (deposit_address = "0x92f29100cc4dca707359d8eb78402eb3acfd87d3", withdraw_address = "0x6cad680b13397a0f1719af6e209d5dd2d228fa4f")
 (deposit_address = "0xf3a6e54248f51fdf85008e02f3ada2dfc66c6c24", withdraw_address = "0x6fa0f8cc1501756bc40c06b841fc908045a1bd7f")
 (deposit_address = "0x06b1bf28c962363f212878bdf87417ebd0316220", withdraw_address = "0xc82547fc22a11b52d2507e28ace078a187194956")
 (deposit_address = "0xc47e04fa576be089a742aa38a2f1215b01

In [74]:
t2 = (deposit_address = "0x1c6558bb4f4dda8fad8c42cc5b0fb69f1c9115a1", withdraw_address = "0x4cb7e575aa19dfc998120a9b056b3b13ca7b0e1c")
d[t2]

Dict{Any, Any} with 2 entries:
  "1 ETH"   => (deposit_hashes = ["0xd1f0e2026d268c855c522332f7777228206f27ae7c…
  "0.1 ETH" => (deposit_hashes = ["0xc103932505443a4efefb4c0500af92b6b4187a7a7f…

In [76]:
d[t2]["0.1 ETH"]

(deposit_hashes = ["0xc103932505443a4efefb4c0500af92b6b4187a7a7fa7b8aa794a9a5239b47396", "0x4b446f425a21af3d0c5aaad088392e59024e45ce7535728f48f6b902826e0515"], withdraw_hashes = ["0xcee0c1d85ad881ec7c33e2f5117bca93d58c91c026a84593f8bf6f63ef5a1687"])